<center>
    <h1> ILI-286 - Laboratorio #3 </h1>
    <h2> Simulando númericamente sistemas dinámicos </h2>
</center>


| Nombre | Rol | Email |
| :----- | :-- | :---- |
| Marco Rojas | 201073005-0 | marco.rojaso@alumnos.usm.cl |
| Hernán Vargas | 201073009-3 | hernan.vargas@alumnos.usm.cl |

## Tabla de contenidos
* [Introducción](#intro)
* [Desarrollo y analisís de resultados:](#desarrollo)
  1. [Simulando Sistemas Caóticos](#se1)
  2. [BVP de orden superior](#se2)
* [Concluciones](#Concluciones)
* [Referencias](#Referencias)
* [Anexos](#Anexos) $\leftarrow$ Comenzar ejecución aquí
  * [Métodos implementados](#an1)
  * [Métodos paralelos](#an2)
  * [Visualización de resultados](#an3)
  * [Implementación de diferencias finitas](#an4)
  * [Utilidades](#an5)

<div id='intro' />
## Introducción
Write something

<div id='desarrollo'/>
## Desarrollo y analisís de resultados

<div id='se1'/>
## 1.- Simulando Sistemas Caóticos

### a) Implementación
Ver [anexo 1](#an1)

### b) Ejecución


In [ ]:
condicion_inicial = "algo"
t_f = 100
h   = "maximo permitido"

#animate_lorenz()

### c) Profiling

In [ ]:
t_f = 1000
#timeit

In [ ]:
#memit

### d) Análisis de estabilidad

In [ ]:
hs = [0,1,2,3]

### e) Implementación paralela
Ver [anexo 2](#an2)

### f) Simulación de dos trayetorias

### g) Simulación de veinte trayetorias con RK4

<div id='se2'/>
## 2.- BVP de orden superior

### a) Transformación a IVP acoplado

### b) Propuesta de cuarta derivada y discretización

### c) Implementación de diferencias finitas
Ver [Anexo 4](#an4)

### d) Ejecución del algoritmo

## Conclusiones

## Referencias
<div id='ref1'\> [1] [Título](www.google.com) Títulos sec. *desc*. Revisado xx/xx/xxxx

## Anexos
Comenzamos cargando los modulos necesarios:

In [ ]:
import numpy as np
from scipy import integrate
from matplotlib import pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.colors import cnames
from matplotlib import animation

#import matplotlib
%matplotlib notebook
#%matplotlib nbagg
#import sympy as sp
#import scipy as sy
#Para retornas formulas matemáticas     from IPython.display import display, Math, Latex
#Para transformar las funciones sympy a lambdas     from sympy.utilities.lambdify import lambdify
#Para borrar el output de timeit, quitar en ipython 4.1 y ejecutar timeit con -q.   from IPython.display import clear_output

#Tamaño de los graficos
FIGSIZE = (12, 6)
#Número muy pequeño:
delta = 10**-15
#IGNORAR WARNINGS    import warnings    warnings.filterwarnings('ignore')

<div id='an1'\> 
### Métodos implementados:

In [ ]:
def euler_int(func, y0, t, h):
    ys = np.zeros([y0.shape[0],t.shape[0]])
    ys[:,0] = y0
    for i in range(t.shape[0]-1):
        ys[:,[i+1]] = ys[:,[i]] + h*func(t[i], ys[:,[i]])
    return ys

In [ ]:
def mp_int(func, y0, t, h):
    h2 = h/2
    ys = np.zeros([y0.shape[0],t.shape[0]])
    ys[:,0] = y0
    for i in range(t.shape[0]-1):
        k1 = ys[:,[i]] + h2*func(t[i], ys[:,[i]])
        ys[:,[i+1]] = ys[:,[i]] + h*func(t[i] + h2 , k1)
    return ys

In [ ]:
def RK4_int(func, y0, t, h):
    h2 = h/2
    ys = np.zeros([y0.shape[0],t.shape[0]])
    ys[:,0] = y0
    for i in range(t.shape[0]-1):
        k1 = func(t[i], ys[:,[i]])
        k2 = func(t[i] + h2, ys[:,[i]] + h2*k1)
        k3 = func(t[i] + h2, ys[:,[i]] + h2*k2)
        k4 = func(t[i] + h,  ys[:,[i]] + h*k3)
        ys[:,[i+1]] = ys[:,[i]] + (h/6)*(k1 + 2*(k2 + k3) + k4)
    return ys

In [ ]:
#test 1-D
y0 = np.array([1])
yf = 4
h  = 1
t  = np.linspace(0, yf, yf/h+1)
f  = lambda t,y: np.e**t


print(euler_int(f, y0, t, h))
print(mp_int(f, y0, t, h))
print(RK4_int(f, y0, t, h))
print(f(4,0))

print("\n")

y02 = np.array([1,2])
yf = 4
h  = 1
t  = np.linspace(0, yf, yf/h+1)

f2 = lambda t,y: np.array([y[0]+t, y[1]+y[0]])

print("Euler")
print(euler_int(f2, y02, t, h))
print("MP")
print(mp_int(f2, y02, t, h))
print("RK4")
print(RK4_int(f2, y02, t, h))
print(f2(4,[0,1]))

<div id='an2'\> 
### Métodos paralelos:

In [ ]:
def parallel_euler_int(func, y0, t, h):
    pass

In [ ]:
def parallel_mp_int(func, y0, t, h):
    pass

In [ ]:
def parallel_RK4_int(func, y0, t, h):
    pass

<div id='an3'\> 
### Visualización de resultados

In [ ]:
"""
-> x_t
   arreglo con las posiciones (x,y,z) de las trayectorias. Este puede ser
   de dos dimensiones (len(t), 3) para una sola trayectoria, o puede ser 
   de tres dimensiones (len(t), 3, N_trajectories) para mas de una trayectoria
-> N_trajectories
   numero de trayectorias a visualizar
-> xlim,ylim,zlim
   limites en los ejes x,y,z de la animacion
-> rotate
   rotar mientras anima para mejores perspectivas
"""
def animate_lorenz(x_t, N_trajectories, xlim=(-20,70), ylim=(-35,35), zlim=(-35,35), rotate=False):
    #setting it to correct format
    if x_t.ndim==2:
        x_t = np.array([x_t])
    elif x_t.ndim==3: 
        x_t = np.rot90(x_t).T
    else:
        return -1
    #setting the number of frames
    frames = max(x_t.shape)
    frames /= 2
    #set up figure & 3D axis for animation
    fig = plt.figure()
    ax = fig.add_axes([0, 0, 1, 1], projection='3d')
    #ax.axis('off')

    #choose a different color for each trajectory
    colors = plt.cm.jet(np.linspace(0, 1, N_trajectories))

    #set up lines and points
    lines = sum([ax.plot([], [], [], '-', c=c)
                 for c in colors], [])
    pts = sum([ax.plot([], [], [], 'o', c=c)
               for c in colors], [])

    #prepare the axes limits
    ax.set_xlim(xlim)
    ax.set_ylim(ylim)
    ax.set_zlim(zlim)

    #set point-of-view: specified by (altitude degrees, azimuth degrees)
    ax.view_init(30, 0)

    #initialization function: plot the background of each frame
    def init():
        for line, pt in zip(lines, pts):
            line.set_data([], [])
            line.set_3d_properties([])

            pt.set_data([], [])
            pt.set_3d_properties([])
        return lines + pts

    #animation function.  This will be called sequentially with the frame number
    def animate(i):
        # we'll step two time-steps per frame.  This leads to nice results.
        i = (2 * i) % x_t.shape[1]

        for line, pt, xi in zip(lines, pts, x_t):
            x, y, z = xi[:i].T
            line.set_data(x, y)
            line.set_3d_properties(z)

            pt.set_data(x[-1:], y[-1:])
            pt.set_3d_properties(z[-1:])

        if rotate: ax.view_init(30, 0.3 * i)
        fig.canvas.draw()
        return lines + pts

    #instantiate the animator.
    anim = animation.FuncAnimation(fig, animate, init_func=init,
                                   frames=frames, interval=30, blit=True, repeat=False)
    #show it
    ax.set_title('Lorenz Simulation')
    ax.set_xlabel('x(t)')
    ax.set_ylabel('y(t)')
    ax.set_zlabel('z(t)')
    plt.show()

<div id='an4'\> 
### Implementación de diferencias finitas

<div id='an5'\> 
### Utilidades